<a href="https://colab.research.google.com/github/robertzak133/2022-Adaptive-Computing/blob/main/Retrain_Yolo_Model_Customer_Data_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transfer Learning Retraining of Yolov3 object Detector
In which I: load a pre-trained version of Yolov3; train it with proivate dataset of labeled trail camear photos; and output it for Quantization an Compilation with Xilinx Tools.  Based on outline in [YOLOv3-Custom-Object-Detection](https://github.com/NSTiwari/YOLOv3-Custom-Object-Detection)

## Loading Data
I am using a dataset comprising trail camera photos taken before and during 2020 and automatically annoted using [Microsoft Megadetector](https://github.com/microsoft/CameraTraps/blob/main/megadetector.md).  Note that there are three classes: animal, people, vehicles.  Note that Megadetector is very good, but not perfect, so some of the images are, in fact known to be mislabeled. 

### Mount GoogleDrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Training using Darknet; Export using Cafee

See "tutorial" at https://github.com/Xilinx/Vitis-AI-Tutorials/tree/1.3/Design_Tutorials/07-yolov4-tutorial

## Create the Dataset in the Darknet Format
We have a hierarchical set of high resolution files, and a json file which describes all the labels therein.   What we need is a flat set of all files in which each <image>.jpg comes with an <image>.txt.  

We're going to sample out of the space to get a desired data set size.  We'll convert all the photos to 800x800 pixels while we're at it. We also need to change the BBOX format

### 416 Resolution for YOLOv3

In [ ]:
!rm -r /content/drive/MyDrive/WBWL2020/res416/
!mkdir /content/drive/MyDrive/WBWL2020/res416/
!mkdir /content/drive/MyDrive/WBWL2020/res416/training
!mkdir /content/drive/MyDrive/WBWL2020/res416/validation


### 608 Resolution for YOLVv4

In [ ]:
!rm -r /content/drive/MyDrive/WBWL2020/res608/


In [ ]:
!mkdir /content/drive/MyDrive/WBWL2020/res608
!mkdir /content/drive/MyDrive/WBWL2020/res608/training
!mkdir /content/drive/MyDrive/WBWL2020/res608/validation

## Utility Code for Creating a Sample of Image Set
The dataset has about 13K photos, many of which are similar.  Instead of training on the whole set, I take a random sample of configurable size.  This code also converts the .json file produced by MegaDetector and converts it to the format required by the darknet training tools.

In [ ]:
import random
import string
import json
import PIL
from PIL import Image
import os
from os.path import exists

In [ ]:
# Convert a megadector bbox list [x_min, y_min, width_of_box, height_of_box] to
#    [center_x, center_y, width_of_box, heigh_of_box] 
def md_to_dn_bbox(x_min, y_min, width_of_box, height_of_box):
  center_x = (x_min + width_of_box) / 2.0
  center_y = (y_min + height_of_box) / 2.0
  return (center_x, center_y, width_of_box, height_of_box)

md_to_dn_bbox(0.1, 0.2, 0.5, 0.4)

In [ ]:
# create a random name for the image comprising two fields of lower case hex
#    digits and a suffix
#    zak 2022-03-21: TODO
#        what I really need is a unique name, but I'm going to let that slide
def create_random_name(length1, length2):
  lc_hex = "0123456789abcdef"
  result_str = ''.join(random.choice(lc_hex) for i in range(length1))
  result_str += '-'
  result_str += ''.join(random.choice(lc_hex) for i in range(length2))
  return result_str

create_random_name(5,5)

In [ ]:
# Convert a json file to a python dictionary
def json_to_python_dict(json_file):
  # Open JSON file 
  f = open(json_file)
  # returns JSON object as
  # a dictionary
  p_dict = json.load(f)
  f.close()
  return p_dict

#json_to_python_dict("/content/drive/MyDrive/WBWL2020/colab_meta_data.json")

In [ ]:
# Returns the total number of images to sample from a given json dictionary
def get_image_indices(p_dict, training_images, validation_fraction):
  total_images = len(p_dict['images'])
  selected_images = 0
  index_list = ()
  if total_images > training_images * (1.0+validation_fraction):
    # we use a fraction of the data set 
    selected_images = int(training_images * (1.0 +validation_fraction))
    print("get_image_indices: selected_images = "+ str(selected_images))
    index_list = random.sample(range(0, total_images), selected_images)
  else:
    # we have to use the whole data set
    selected_images = total_images
    index_list = range(0, total_images)
  return (index_list)

In [ ]:
# Write all the images in a given list of indices
def write_image_meta_data(name, j_dict, index_list, output_dir, image_dimension):
  # Loop through the dictionary for everything in the list
  directory_path = os.path.join(output_dir, name)
  count = 0
  for image_index in index_list:
    random_filename = create_random_name(5,5)
    image_descriptor = j_dict['images'][image_index]
    image_filename = image_descriptor['file']
    if not(exists(image_filename)):
      print("Aborting " + str(image_index) + ": file does not exist")
      continue
    if os.path.getsize(image_filename) < 10: 
      print("Aborting " + str(image_index) + ": file is truncated")
      continue
    count= 1+ count
    print(str(count) + ':' + image_filename)
    # open the image file and write it out resized
    image = Image.open(image_filename)
    image = image.resize((image_dimension, image_dimension))
    image.save(os.path.join(directory_path, random_filename + ".jpg"))
    # write out a meta-data file
    meta_data_filename = os.path.join(directory_path, random_filename + '.txt')
    with open(meta_data_filename, 'w',encoding = 'utf-8') as mdf:
      for detection in image_descriptor['detections']:
        ## Yolo wants a zero-based category ID -- I don't know how it handles empty images
        category = int(detection['category']) - 1
        xmin, ymin, xsize, ysize = detection['bbox']
        xcent, ycent, width, height = md_to_dn_bbox(xmin, ymin, xsize, ysize)
        mdf.write(str(category) + ' ' + str(xcent) + ' '+str(ycent) + ' ' + str(width) + ' ' +str(height) + '\n')
  return

In [ ]:
# Write Validation and Training Images and Meta Data
def write_images_metadata(p_dict, output_dir, training_images, validation_fraction, image_dimension):
  # get the number total number of images we'll use
  total_index_list = get_image_indices(p_dict, training_images, validation_fraction)
  num_val_images = int(len(total_index_list) * (validation_fraction/(1.0+ validation_fraction)))
  print (num_val_images)
  val_index_list = total_index_list[:num_val_images]
  training_index_list = total_index_list[num_val_images:]
  print('Writing Validation Dataset')
  write_image_meta_data('validation', p_dict, val_index_list, output_dir, image_dimension)
  print('Writing Training Dataset')
  write_image_meta_data('training', p_dict, training_index_list, output_dir, image_dimension)
  return

In [ ]:
# Extract 
def json_to_darknet(json_file, output_dir, max_training_images, validation_fraction, image_dimension):
  print("json_to_darknet")
  print("   json_file  = " + json_file)
  print("   output_dir = " + output_dir)
  print("   max_training_images = " + str(max_training_images))
  print("   validation_fraction = " + str(validation_fraction))
  print("   image_dimension     = " + str(image_dimension))
  # 
  j_dict = json_to_python_dict(json_file)
  write_images_metadata(j_dict, output_dir, max_training_images, validation_fraction, image_dimension)  
  return




### Create Resolution 416 Image List

In [ ]:
json_to_darknet("/content/drive/MyDrive/WBWL2020/colab_meta_data.json", "/content/drive/MyDrive/WBWL2020/res416/", 2000, 0.20, 416)

### Create Resolution 608 Image List


In [ ]:
json_to_darknet("/content/drive/MyDrive/WBWL2020/colab_meta_data.json", "/content/drive/MyDrive/WBWL2020/res608/", 2500, 0.20, 608)

## YOLOv3-Custom-Object-Detection
From https://github.com/NSTiwari/YOLOv3-Custom-Object-Detection/blob/main/YOLOv3_Custom_Object_Detection.ipynb

### 1. Clone, configure & compile Darknet

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Compile
!make

### 2. Configure yolov3.cfg file

In [ ]:
# Make a copy of yolov3.cfg
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
# Change lines in yolov3.cfg file
# note filters = 3 * (classes + 5)
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 6000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=3@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=24@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=24@' cfg/yolov3_training.cfg

In [ ]:
!cp cfg/yolov3_training.cfg cfg/yolov3_testing.cfg

In [ ]:
!grep classes cfg/yolov3_testing.cfg

### 3. Create .names and .data files

In [ ]:
!echo -e 'Animal\nPerson\nVehicle' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/Models/yolov3_ct' > data/obj.data


### 4. Save yolov3_training.cfg and obj.names files in Google Drive

In [ ]:
!cp cfg/yolov3_training.cfg /content/drive/MyDrive/Models/yolov3_ct/yolov3_training.cfg
!cp cfg/yolov3_testing.cfg /content/drive/MyDrive/Models/yolov3_ct/yolov3_testing.cfg
!cp data/obj.names /content/drive/MyDrive/Models/yolov3_ct/classes.txt

### 5. Create a Local Folder for image dataset

In [ ]:
!ls /content/drive/MyDrive/WBWL2020/res416/training

In [ ]:
!rm -r data/obj
!mkdir data/obj
!cp /content/drive/MyDrive/WBWL2020/res416/training/* data/obj/
!cp /content/drive/MyDrive/WBWL2020/res416/validation/* data/obj/

### 6. Create train.txt and valid.txt files

In [ ]:
import glob
images_list = glob.glob("/content/drive/MyDrive/WBWL2020/res416/training/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
images_list = glob.glob("/content/drive/MyDrive/WBWL2020/res416/validation/*.jpg")
with open("data/test.txt", "w") as f:
  f.write("\n".join(images_list))

In [ ]:
!head data/train.txt

In [ ]:
!head data/test.txt

### 7. Download pre-trained wieght for the convoluational layers file

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

### 8. Start Training

In [ ]:
#!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
!./darknet detector train data/obj.data cfg/yolov3_training.cfg /content/drive/MyDrive/Models/yolov3_ct/yolov3_training_last.weights -dont_show

### 9. Testing Resulting Model
In which I run the validation set through the model to see how it works. 

In [ ]:
!rm -r results
!mkdir results


In [ ]:
!./darknet detector map data/obj.data cfg/yolov3_testing.cfg /content/drive/MyDrive/Models/yolov3_ct/yolov3_training_final.weights -dont_show

### 10. Converting the Model to .xmodel format 
I (eventually) tried slightly modified versions of Xilinx Conversion scripts outlined in the "caffe" route through quantization and compilation:  
https://github.com/Xilinx/Vitis-AI-Tutorials/tree/1.3/Design_Tutorials/07-yolov4-tutorial

This work is continued in a platform (unlike colab) which supports docker and Vitis-AI toolset.  See you there. 

## YOLOv4 Custom Object Detection
Following outline https://github.com/Xilinx/Vitis-AI-Tutorials/tree/1.3/Design_Tutorials/07-yolov4-tutorial

This is work in progress

### 1. Clone, configure & compile Darknet

In [ ]:
# Clone
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# Configure
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile

In [ ]:
# Compile
!make

In [ ]:
# Check if NVIDIA GPU is Enabled
!nvidia-smi

### 2. Configure yolov4.cfg file

In [ ]:
# Make a copy of yolov4.cfg
!cp cfg/yolov4.cfg cfg/yolov4_training.cfg

In [ ]:
# Change lines in yolov4.cfg file
# Change the size of the input
#!sed -i 's/width=608/width=416/' cfg/yolov4_training.cfg
#!sed -i 's/height=608/height=416/' cfg/yolov4_training.cfg
# "the MISH activation layers are swapped to leaky as the DPU doesn't support MISH."
!sed -i 's/activation=mish/activation=leaky/' cfg/yolov4_training.cfg
# "I saw the best post Quantization accuracy results when commenting out the 
#  3 max_pooling layers as follows. Note the route layers were changed to 
#  -1,-3,-4,-6 from -1,-3,-5,-6"
!sed -i '771 s@\[maxpool\]@#\[maxpool\]@' cfg/yolov4_training.cfg
!sed -i '772 s@stride=1@#stride=1@' cfg/yolov4_training.cfg
!sed -i '773 s@size=5@#size=5@' cfg/yolov4_training.cfg
!sed -i '775 s@\[route\]@#\[route\]@' cfg/yolov4_training.cfg
!sed -i '776 s@layers=-2@#layers=-2@' cfg/yolov4_training.cfg
#
!sed -i '778 s@\[maxpool\]@#\[maxpool\]@' cfg/yolov4_training.cfg
!sed -i '779 s@stride=1@#stride=1@' cfg/yolov4_training.cfg
!sed -i '780 s@size=9@#size=9@' cfg/yolov4_training.cfg
!sed -i '782 s@\[route\]@#\[route\]@' cfg/yolov4_training.cfg
!sed -i '783 s@layers=-4@#layers=-4@' cfg/yolov4_training.cfg
#
!sed -i '785 s@\[maxpool\]@#\[maxpool\]@' cfg/yolov4_training.cfg
!sed -i '786 s@stride=1@#stride=1@' cfg/yolov4_training.cfg
!sed -i '787 s@size=13@#size=13@' cfg/yolov4_training.cfg
!sed -i '789 s@\[route\]@#\[route\]@' cfg/yolov4_training.cfg
!sed -i '790 s@layers=-1,-3,-5,-6@#layers=-1,-3,-4,-6@' cfg/yolov4_training.cfg
# Other Changes
# !sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov4_training.cfg
!sed -i 's/max_batches = 500500/max_batches = 6000/' cfg/yolov4_training.cfg
!sed -i '968 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '1056 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '1144 s@classes=80@classes=3@' cfg/yolov4_training.cfg
!sed -i '961 s@filters=255@filters=24@' cfg/yolov4_training.cfg
!sed -i '1049 s@filters=255@filters=24@' cfg/yolov4_training.cfg
!sed -i '1137 s@filters=255@filters=24@' cfg/yolov4_training.cfg

In [ ]:
!head cfg/yolov4_training.cfg

In [ ]:
!cp cfg/yolov4_training.cfg cfg/yolov4_testing.cfg

In [ ]:
!grep classes cfg/yolov4_testing.cfg

### 3. Create .names and .data files

In [ ]:
!echo -e 'Animal\nPerson\nVehicle' > data/obj.names
!echo -e 'classes= 3\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /content/drive/MyDrive/Models/yolov4' > data/obj.data


### 4. Save yolov4_training.cfg and obj.names files in Google Drive

In [ ]:
!mkdir /content/drive/MyDrive/Models/yolov4

In [ ]:
!cp cfg/yolov4_training.cfg /content/drive/MyDrive/Models/yolov4/yolov4_training.cfg
!cp cfg/yolov4_training.cfg /content/drive/MyDrive/Models/yolov4/yolov4_testing.cfg
!cp data/obj.names /content/drive/MyDrive/Models/yolov4/classes.txt

### 5. Create a Local Folder for image dataset

In [ ]:
!rm -r data/obj
!mkdir data/obj
!cp /content/drive/MyDrive/WBWL2020/res608/training/* data/obj/
!cp /content/drive/MyDrive/WBWL2020/res608/validation/* data/obj/

### 6. Create train.txt and train.txt files

In [ ]:
import glob
images_list = glob.glob("/content/drive/MyDrive/WBWL2020/res608/training/*.jpg")
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))

In [ ]:
images_list = glob.glob("/content/drive/MyDrive/WBWL2020/res608/validation/*.jpg")
with open("data/test.txt", "w") as f:
  f.write("\n".join(images_list))

In [ ]:
!head data/train.txt

### 7. Download pre-trained wieght for the convoluational layers file

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.weights

### 8. Start Training

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4_training.cfg yolov4.weights -dont_show
# Uncomment below and comment above to re-start your training from last saved weights
#!./darknet detector train data/obj.data cfg/yolov4_training.cfg /content/drive/MyDrive/Models/yolov4/yolov4_training_last.weights -dont_show

### 9. Testing Resulting Model
In which I run the validation set through the model to see how it works. 

In [ ]:
!rm -r results
!mkdir results


In [ ]:
!./darknet detector map data/obj.data cfg/yolov4_training.cfg /content/drive/MyDrive/Models/yolov4/yolov4_training_final.weights -dont_show

#Archived -- Does not Work
We're going to try to do this all in tensorflow 1.x 

2022-03-25: You can't start from a "real" yolov3 model.  The real models contain constructs that don't work on the Xilinx DPU.  Thus, you have to modify the model.  Once having appreciated this, I'm going back to a path which seemed to *almost* work.  Saving this for archive.

We'll start by following https://github.com/YunYang1994/tensorflow-yolov3.git

I found that I wanted to change some of the files in this repository, so I cloned myself a copy to modify at: https://github.com/robertzak133/tensorflow-yolov3.git

```
# This is formatted as code
```




### Set up environment, load models


In [ ]:
!git clone https://github.com/robertzak133/tensorflow-yolov3.git

In [ ]:
# according to https://stackoverflow.com/questions/57677160/cannot-import-name-keras-export-from-tensorflow-python-util-tf-export 
# this was why original was getting errors
!pip uninstall -y tensorflow

In [ ]:
%cd tensorflow-yolov3
# note that I had to hack in a slightly later version of tensorflow
# !sed -i 's/tensorflow-gpu==1.11.0/tensorflow-gpu==1.13.1/' ./docs/requirements.txt
!pip install -r ./docs/requirements.txt

In [ ]:
!more ./docs/requirements.txt

In [ ]:
%cd checkpoint
!wget https://github.com/YunYang1994/tensorflow-yolov3/releases/download/v1.0/yolov3_coco.tar.gz
!tar -xvf yolov3_coco.tar.gz


In [ ]:
%cd ..
!python convert_weight.py
!python freeze_graph.py

In [ ]:
!ls


In [ ]:
# Copy resulting model to google drive so I can verify that the 
#      baseline model (before I start mucking with it) quantizes
#      with Xilinx tools
!cp yolov3_coco.pb /content/drive/MyDrive/Models/yolov3_coco/

In [ ]:
# tweak image_demo.py for colab environment
!sed -i '52 s/image/image(convert)/' ./image_demo.py

In [ ]:
!tail image_demo.py

In [ ]:
# Test out model on demo image
# (runs, but display doesn't seem to work on Colab)
!python image_demo.py

In [ ]:
%tensorflow_version 1.x
import tensorflow
print(tensorflow.__version__)

In [ ]:
pip install mmdnn

In [ ]:
!mmtoir -f darknet -n /content/drive/MyDrive/Models/yolov3/yolov3_testing.cfg -w /content/drive/MyDrive/Models/yolov3/yolov3_training_final.weights -o /content/drive/MyDrive/Models/yolov3/darknet_yolov3 --darknetStart 0

In [ ]:
!ls -alst /content/drive/MyDrive/Models/yolov3/